In [1]:
%%time
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
from snowflake.ml.registry.registry import Registry
import pandas as pd

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
CPU times: user 962 ms, sys: 605 ms, total: 1.57 s
Wall time: 11.9 s


In [2]:
%%time
monitor_resp = [
    {
        "run_id": "8042932c-101a-4ddc-a808-00c092797960",
        "algorithm_used": "XGBClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "16e3f179-bb70-4632-9a39-9a6fac4a5581",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "ef47a0a5-1a74-4a14-b0fd-8dbbd322057f",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running",
        "model_metrics": {}
    },
    {
        "run_id": "26a15f9c-29fd-4d55-82fb-3d38cc18e4b8",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running",
        "model_metrics": {}
    }
]

def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
        
session = get_session("EMPLOYEE_10L","0e0fb803-22db-4d88-9f2f-f6f75b6abcf0")
reg = Registry(session=session)

model_list = session.sql("show models like '%exp6789%'").collect()
for model in model_list:
    model_name = model["name"]
    model_run_id = model_name[-36:].replace('_','-').lower()
    model_obj = reg.get_model(model_name)
    metrics = {
        "model_metrics" : model_obj.default.get_metric('model_metrics')
    }
    for run in monitor_resp:
        if run.get('run_id') == model_run_id:
            run.update(metrics)
print(monitor_resp)

[{'run_id': '8042932c-101a-4ddc-a808-00c092797960', 'algorithm_used': 'XGBClassifier', 'status': 'Successful', 'model_metrics': {'accuracy_score': 0.912834, 'f1_score': 0.863711823763472, 'recall_score': 0.8009236633760141, 'precision_score': 0.9371819065591582, 'roc_auc': 0.8863324874130605, 'log_loss': 3.1437201566897612}}, {'run_id': '16e3f179-bb70-4632-9a39-9a6fac4a5581', 'algorithm_used': 'DecisionTreeClassifier', 'status': 'Successful', 'model_metrics': {'accuracy_score': 0.92473, 'f1_score': 0.8841251148525358, 'recall_score': 0.8361769313619897, 'precision_score': 0.9379067021773153, 'roc_auc': 0.9036114375207266, 'log_loss': 2.7149916732535027}}, {'run_id': 'ef47a0a5-1a74-4a14-b0fd-8dbbd322057f', 'algorithm_used': 'GradientBoostingClassifier', 'status': 'Running', 'model_metrics': {}}, {'run_id': '26a15f9c-29fd-4d55-82fb-3d38cc18e4b8', 'algorithm_used': 'RandomForestClassifier', 'status': 'Running', 'model_metrics': {}}]
CPU times: user 793 ms, sys: 239 ms, total: 1.03 s
Wall 

In [3]:
%%time
####Approach 2####
monitor_resp = [
    {
        "run_id": "8042932c-101a-4ddc-a808-00c092797960",
        "algorithm_used": "XGBClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "16e3f179-bb70-4632-9a39-9a6fac4a5581",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "ef47a0a5-1a74-4a14-b0fd-8dbbd322057f",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running",
        "model_metrics": {}
    },
    {
        "run_id": "26a15f9c-29fd-4d55-82fb-3d38cc18e4b8",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running",
        "model_metrics": {}
    }
]

def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
        
session = get_session("EMPLOYEE_10L","0e0fb803-22db-4d88-9f2f-f6f75b6abcf0")
reg = Registry(session=session)
resp_df = pd.DataFrame(monitor_resp)
model_list = session.sql("show models like '%exp6789%'").collect()
for model in model_list:
    model_name = model["name"]
    model_run_id = model_name[-36:].replace('_','-').lower()
    model_obj = reg.get_model(model_name)
    metrics = {
        "model_metrics" : model_obj.default.get_metric('model_metrics')
    }
    select_row = resp_df.loc[resp_df['run_id']==model_run_id]
    resp_df.loc[resp_df['run_id']==model_run_id, 'model_metrics'] = str(metrics.get('model_metrics'))   
print(resp_df)

                                 run_id              algorithm_used  \
0  8042932c-101a-4ddc-a808-00c092797960               XGBClassifier   
1  16e3f179-bb70-4632-9a39-9a6fac4a5581      DecisionTreeClassifier   
2  ef47a0a5-1a74-4a14-b0fd-8dbbd322057f  GradientBoostingClassifier   
3  26a15f9c-29fd-4d55-82fb-3d38cc18e4b8      RandomForestClassifier   

       status                                      model_metrics  
0  Successful  {'accuracy_score': 0.912834, 'f1_score': 0.863...  
1  Successful  {'accuracy_score': 0.92473, 'f1_score': 0.8841...  
2     Running                                                 {}  
3     Running                                                 {}  
CPU times: user 559 ms, sys: 0 ns, total: 559 ms
Wall time: 3.1 s


In [2]:
import requests

In [22]:
monitor_url = "http://monitor-backend-service/monitor/api/v5/executions/search?objectId=f4c813a1-86f1-4148-9ef3-cf8758331729&projectId=784a85f3-443c-4662-b56d-20d9bccd0027&&executedBy=aditya1.singh2@fosfor.com"
temp_head = {
    "X-Auth-Userid": "aditya1.singh2@fosfor.com",
    "X-Auth-Username": "aditya1.singh2@fosfor.com",
    "X-Auth-Email": "aditya1.singh2@fosfor.com",
    "X-Project-Id": "0e0fb803-22db-4d88-9f2f-f6f75b6abcf0"
}

In [23]:
res = requests.get(monitor_url, headers=temp_head)

In [24]:
res.

'{"totalRecordCount":19,"data":[{"jobinstanceId":499867,"objectId":"f4c813a1-86f1-4148-9ef3-cf8758331729","projectId":"784a85f3-443c-4662-b56d-20d9bccd0027","projectName":"Experiments","jobObjectName":"sample_Airline_RandomForestC_10k.ipynb","jobScheduleId":null,"jobInstanceInputParameter":"{job:FIRST,jobType:Notebook}","jobType":"SCHEDULE","jobStatus":"FAILED","finalStatus":"FAILED","jobCreatedby":"aditya1.singh2@fosfor.com","jobCreatedts":"2024-07-07T18:30:02.201+00:00","jobUpdatedby":"aditya1.singh2@fosfor.com","jobUpdatedts":"2024-07-07T18:31:02.466+00:00","applicationId":null,"executionId":"77305","runID":"499867","versionNumber":"","runEngine":null,"attemptId":0,"workflowId":"","jobConfig":null},{"jobinstanceId":495247,"objectId":"f4c813a1-86f1-4148-9ef3-cf8758331729","projectId":"784a85f3-443c-4662-b56d-20d9bccd0027","projectName":"Experiments","jobObjectName":"sample_Airline_RandomForestC_10k.ipynb","jobScheduleId":null,"jobInstanceInputParameter":"{job:FIRST,jobType:Notebook}"